## Importamos y exploramos los datos

In [1]:
!cd review_polarity_competition/

In [2]:
!ls review_polarity_competition/reviews_sentoken/neg

1000.txt  532.txt  599.txt  666.txt  733.txt  800.txt  867.txt	934.txt
1001.txt  533.txt  600.txt  667.txt  734.txt  801.txt  868.txt	935.txt
1002.txt  534.txt  601.txt  668.txt  735.txt  802.txt  869.txt	936.txt
1003.txt  535.txt  602.txt  669.txt  736.txt  803.txt  870.txt	937.txt
1004.txt  536.txt  603.txt  670.txt  737.txt  804.txt  871.txt	938.txt
1005.txt  537.txt  604.txt  671.txt  738.txt  805.txt  872.txt	939.txt
1006.txt  538.txt  605.txt  672.txt  739.txt  806.txt  873.txt	940.txt
1007.txt  539.txt  606.txt  673.txt  740.txt  807.txt  874.txt	941.txt
1008.txt  540.txt  607.txt  674.txt  741.txt  808.txt  875.txt	942.txt
1009.txt  541.txt  608.txt  675.txt  742.txt  809.txt  876.txt	943.txt
1010.txt  542.txt  609.txt  676.txt  743.txt  810.txt  877.txt	944.txt
1011.txt  543.txt  610.txt  677.txt  744.txt  811.txt  878.txt	945.txt
1012.txt  544.txt  611.txt  678.txt  745.txt  812.txt  879.txt	946.txt
1013.txt  545.txt  612.txt  679.txt  746.txt  813.txt  880.txt	947.txt
1014.t

In [3]:
%load_ext autoreload
%autoreload 2

In [5]:
import pandas as pd
import numpy as np

In [4]:
!cat review_polarity_competition/reviews_sentoken/neg/1010.txt

the movie sucks ... just like that . Got to say , photography , customes , and the sets are good , but the story is really lame ... also , the action sequences are crappy ( almos funny ) ... and the end.. uffff ... please ... I just fast forwarded the movie so it would end sooner ... I really , really like the game ... but the movie sucks

In [6]:
from sklearn.datasets import load_files
dataset = load_files('review_polarity_competition/reviews_sentoken/', shuffle=False)

In [7]:
dataset.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [8]:
df = pd.DataFrame({
    'data': dataset['data'],
    'target': dataset['target']
})
df.head()

,data,target
0,"b""Nothing new happens in this movie . It has p...",0
1,b'Agree with the others . Was very excited to ...,0
2,"b""I am a huge fan of Christmas shows , new and...",0
3,"b""`` Battlefield Earth '' is rightly considere...",0
4,"b""I have seen and loved all three of these mov...",0


In [9]:
# Chequeamos que las clases esten balanceadas
df.target.mean()

0.5

## Empezamos con algunos modelos basicos como baselines

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

vect = CountVectorizer(min_df=5, max_features=100)
vect.fit(df.data)
X = vect.transform(df.data)
vect_TFIDF = TfidfVectorizer(binary=True)
vect_TFIDF.fit(df.data)
X_TF = vect_TFIDF.transform(df.data)
y = df.target

In [13]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier

clfs = [
    KNeighborsClassifier(),
    MultinomialNB(),
    DecisionTreeClassifier(random_state=0),
    LogisticRegression(random_state=0),
    LinearSVC(random_state=0),
    SVC(random_state=0),
    RandomForestClassifier(random_state=0),
]

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, cross_val_predict
from util import print_short_eval

#scores = dict(zip([str(type(clf).__name__) for clf in clfs], [0 for i in clfs]))
scores =[]

for clf in clfs:
    scores.append(cross_val_score(clf, X, y, cv=3))

/home/gaston/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/gaston/miniconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/gaston/miniconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/gaston/miniconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to 

In [15]:
scores

[array([0.59217877, 0.65449438, 0.62640449]),
 array([0.70391061, 0.6994382 , 0.69662921]),
 array([0.62849162, 0.62359551, 0.63202247]),
 array([0.70391061, 0.6994382 , 0.69101124]),
 array([0.70111732, 0.70505618, 0.68258427]),
 array([0.67318436, 0.73033708, 0.73033708]),
 array([0.6424581 , 0.65730337, 0.67134831])]

In [16]:
scores_tf =[]

for clf in clfs:
    scores_tf.append(cross_val_score(clf, X_TF, y, cv=3).mean())

/home/gaston/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/gaston/miniconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/gaston/miniconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/gaston/miniconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale'

In [17]:
scores_tf

[0.7916222040884647,
 0.8402276484003934,
 0.6541752976377294,
 0.8402276484003934,
 0.8392651643546126,
 0.7458048249743685,
 0.7224384324064194]

In [18]:
# gano la logistic
from sklearn.pipeline import Pipeline

clf = LogisticRegression()
clf.fit(X_TF, y)


/home/gaston/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

## Validar

In [ ]:
import os
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split

def load_datasets_unlabeled_test():
    dataset = load_files('review_polarity_competition/reviews_sentoken', shuffle=False)
    docs_train, docs_dev, y_train, y_dev = train_test_split(
        dataset.data, dataset.target, test_size=0.10, random_state=42)
    dirname = "review_polarity_competition/test_reviews_sentoken"
    test = []
    # I do this to keep the files in numeric order
    for fname in range(len(os.listdir(dirname))):
        fname = str(fname) + ".txt"
        with open(os.path.join(dirname, fname)) as fd:
            test.append((fd.read(), fname))
    train = docs_train, y_train
    dev = docs_dev, y_dev
    return train, dev, test

def save_results(fname, labels):
    with open(fname, 'w') as f:
        f.write("Id,Category\n")
        for i,l in enumerate(labels):
            f.write(str(i) + ".txt," + str(l) + "\n")

In [ ]:
train, dev, test = load_datasets_unlabeled_test()

In [ ]:
X_test = vect_TFIDF.transform(test)
y_test_pred = clf.predict(X_test)

In [ ]:
test_df = pd.DataFrame(test)
test_df.head()

,0,1
0,The music track of this version is made up of ...,0.txt
1,I gave this to my 4 year and 2 year old grandc...,1.txt
2,i was n't expecting much from this dvd ... .bu...,2.txt
3,I 've seen this one lots of times . I believe ...,3.txt
4,I purchased all the Inspector Morse DVDs and h...,4.txt


In [ ]:
test_df[1]

0        0.txt
1        1.txt
2        2.txt
3        3.txt
4        4.txt
5        5.txt
6        6.txt
7        7.txt
8        8.txt
9        9.txt
10      10.txt
11      11.txt
12      12.txt
13      13.txt
14      14.txt
15      15.txt
16      16.txt
17      17.txt
18      18.txt
19      19.txt
20      20.txt
21      21.txt
22      22.txt
23      23.txt
24      24.txt
25      25.txt
26      26.txt
27      27.txt
28      28.txt
29      29.txt
        ...   
470    470.txt
471    471.txt
472    472.txt
473    473.txt
474    474.txt
475    475.txt
476    476.txt
477    477.txt
478    478.txt
479    479.txt
480    480.txt
481    481.txt
482    482.txt
483    483.txt
484    484.txt
485    485.txt
486    486.txt
487    487.txt
488    488.txt
489    489.txt
490    490.txt
491    491.txt
492    492.txt
493    493.txt
494    494.txt
495    495.txt
496    496.txt
497    497.txt
498    498.txt
499    499.txt
Name: 1, Length: 500, dtype: object

In [ ]:
dirname = "review_polarity_competition/test_reviews_sentoken"
save_results('/home/results.csv', y_test_pred)